<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%BF%D0%BE%D0%B3%D1%80%D0%B5%D1%88%D0%BD%D0%BE%D1%81%D1%82%D0%B8%20(wall%3F).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab-ready script: OKX scanner -> Telegram signals
!pip install --quiet ccxt pandas numpy requests

import time
import datetime as dt
import requests
import pandas as pd
import numpy as np
import ccxt

# ---------------- CONFIG ----------------
CONFIG = {
    "telegram_bot_token": "8111633898:AAEH_ssxu0KiyTNHbZ2Ig04ilSTCRGCV6b8",
    "telegram_chat_id": "-1003024991695",
    "symbols": [],  # пустой список = динамический подбор всех USDT
    "market_filter": {"quote": "USDT", "type_contains": ["SWAP","PERPETUAL","PERP"]},
    "scan_interval": 10*60,
    "delta_window_trades": 200,
    "delta_ratio_threshold": 0.6,
    "vwap_window_minutes": 60,
    "atr_period": 14,
    "pivot_anchor_tf": "1h",
    "near_level_atr_mult": 0.5,
    "min_confidence_score": 6.0,
    "min_base_volume_24h": 100000
}

# ---------------- helpers ----------------
def send_telegram_message(bot_token, chat_id, text, parse_mode="HTML"):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": parse_mode, "disable_web_page_preview": True}
    try:
        r = requests.post(url, json=payload, timeout=10)
        print("📨 Telegram response:", r.status_code)
        return r.json()
    except Exception as e:
        print("❌ Telegram send error:", e)
        return None

def now_ts():
    return dt.datetime.now(dt.timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

exchange = ccxt.okx({"enableRateLimit": True})

def safe_fetch_ohlcv(symbol, timeframe='1h', limit=200):
    try:
        return exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    except:
        return None

# ---------------- indicators ----------------
def compute_rsi(ohlcv, period=14):
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
    delta = df['close'].astype(float).diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.ewm(alpha=1/period, adjust=False).mean()
    ma_down = down.ewm(alpha=1/period, adjust=False).mean()
    rs = ma_up / (ma_down.replace(0,np.nan))
    rsi = 100 - (100 / (1+rs))
    return float(rsi.iloc[-1]) if not np.isnan(rsi.iloc[-1]) else 50

def compute_ema(series, period):
    s = pd.Series(series).astype(float)
    return float(s.ewm(span=period, adjust=False).mean().iloc[-1]) if len(s)>=2 else None

def compute_macd(series, fast=12, slow=26, signal=9):
    s = pd.Series(series)
    ema_fast = s.ewm(span=fast, adjust=False).mean()
    ema_slow = s.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    hist = macd_line - signal_line
    return macd_line.iloc[-1], signal_line.iloc[-1], hist.iloc[-1]

def compute_bollinger(series, period=20, mult=2.0):
    s = pd.Series(series)
    sma = s.rolling(period).mean().iloc[-1]
    std = s.rolling(period).std().iloc[-1]
    return sma + mult*std, sma - mult*std, sma

def compute_atr(ohlcv, period=14):
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
    df['prev_close'] = df['close'].shift(1)
    df['tr'] = df[['high','low','prev_close']].apply(lambda x: max(x['high']-x['low'], abs(x['high']-x['prev_close']), abs(x['low']-x['prev_close'])), axis=1)
    atr = df['tr'].rolling(period).mean().iloc[-1]
    return float(atr) if not np.isnan(atr) else 0.0

def compute_vwap(ohlcv):
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
    df['typ'] = (df['high']+df['low']+df['close'])/3
    df['pv'] = df['typ']*df['vol']
    return (df['pv'].sum()/df['vol'].sum()) if df['vol'].sum()>0 else None

def get_best_bid_ask(ob):
    bids = ob.get('bids') or []
    asks = ob.get('asks') or []
    return (float(bids[0][0]) if bids else None, float(asks[0][0]) if asks else None)

# ---------------- multi-timeframe analysis ----------------
def analyze_multitimeframe(symbol, tf_list=['1m','15m','30m','4h','12h','24h']):
    trends = []
    ohlcv_dict = {}
    for tf in tf_list:
        ohlcv = safe_fetch_ohlcv(symbol, tf, limit=200)
        if not ohlcv or len(ohlcv)<20:
            continue
        ohlcv_dict[tf] = ohlcv
        df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
        closes = df['close'].astype(float)
        sma20 = closes.rolling(20).mean().iloc[-1]
        sma50 = closes.rolling(50).mean().iloc[-1]
        ema20 = closes.ewm(span=20, adjust=False).mean().iloc[-1]
        ema50 = closes.ewm(span=50, adjust=False).mean().iloc[-1]
        macd_line, signal_line, hist = compute_macd(closes)
        rsi = compute_rsi(ohlcv)
        bullish = (ema20>ema50) and (sma20>sma50) and (hist>0) and (rsi>50)
        bearish = (ema20<ema50) and (sma20<sma50) and (hist<0) and (rsi<50)
        if bullish: trends.append('long')
        elif bearish: trends.append('short')
        else: trends.append('neutral')

    if all(t=='long' for t in trends): consensus='long'
    elif all(t=='short' for t in trends): consensus='short'
    else: consensus='neutral'
    return consensus, ohlcv_dict

# ---------------- scoring ----------------
def score_reversal(symbol, price, ohlcv, trades, ob, cfg):
    atr = compute_atr(ohlcv, cfg['atr_period'])
    vwap = compute_vwap(ohlcv)
    if not vwap or atr==0:
        return None
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
    closes = df['close'].astype(float).tolist()
    # LONG / SHORT определяем по цене и VWAP
    if price < vwap:
        side = 'long'
        entry_price = float(price)
        sl = entry_price - atr       # стоп ниже цены входа
        tp = entry_price + atr*2     # тейк выше цены входа
    else:
        side = 'short'
        entry_price = float(price)
        sl = entry_price + atr       # стоп выше цены входа
        tp = entry_price - atr*2     # тейк ниже цены входа
    score = 7.0  # пока простая фиктивная оценка
    return {"score": score, "side": side, "entry_price": entry_price, "SL": sl, "TP": tp}


# ---------------- scan symbol ----------------
def scan_symbol_multitimeframe(sym, cfg):
    consensus, ohlcv_multi = analyze_multitimeframe(sym)
    if consensus=='neutral': return None
    main_tf = '1h'
    ohlcv_main = ohlcv_multi.get(main_tf) or safe_fetch_ohlcv(sym, main_tf)
    if not ohlcv_main: return None
    try: ticker = exchange.fetch_ticker(sym); last_price=float(ticker['last'])
    except: return None
    try: trades=exchange.fetch_trades(sym, limit=cfg['delta_window_trades'])
    except: trades=[]
    try: ob=exchange.fetch_order_book(sym, limit=100)
    except: ob={'bids':[], 'asks':[]}
    res=score_reversal(sym,last_price,ohlcv_main,trades,ob,cfg)
    res['side']=consensus
    if res['score']>=cfg['min_confidence_score']:
        msg = f"<b>{sym}</b> — {res['side'].upper()}\nEntry: {res['entry_price']:.4f}\nSL: {res['SL']:.4f}\nTP: {res['TP']:.4f}\nScore: {res['score']:.2f}"
        send_telegram_message(cfg['telegram_bot_token'], cfg['telegram_chat_id'], msg)
        return res
    return None

# ---------------- main loop ----------------
def main_loop(cfg):
    symbols = cfg['symbols'] or [s for s in exchange.load_markets() if 'USDT' in s]
    print(f"✅ Found {len(symbols)} filtered symbols")
    while True:
        print(f"{now_ts()} — scanning {len(symbols)} symbols...")
        for i, sym in enumerate(symbols,1):
            try:
                print(f"[{i}/{len(symbols)}] Scanning {sym}...")
                scan_symbol_multitimeframe(sym, cfg)
            except Exception as e:
                print(f"❌ Error scanning {sym}: {e}")
        print(f"{now_ts()} — waiting {cfg['scan_interval']} sec...\n")
        time.sleep(cfg['scan_interval'])

# ---------------- run ----------------
main_loop(CONFIG)


Выходные данные были обрезаны до нескольких последних строк (5000).
[523/547] Scanning UMA/USDT:USDT...
[524/547] Scanning UNI/USDT:USDT...
[525/547] Scanning USDC/USDT:USDT...
[526/547] Scanning USELESS/USDT:USDT...
[527/547] Scanning VANA/USDT:USDT...
[528/547] Scanning VIRTUAL/USDT:USDT...
[529/547] Scanning W/USDT:USDT...
[530/547] Scanning WAL/USDT:USDT...
[531/547] Scanning WCT/USDT:USDT...
[532/547] Scanning WIF/USDT:USDT...
[533/547] Scanning WLD/USDT:USDT...
[534/547] Scanning WOO/USDT:USDT...
[535/547] Scanning XAN/USDT:USDT...
[536/547] Scanning XAUT/USDT:USDT...
[537/547] Scanning XLM/USDT:USDT...
[538/547] Scanning XPL/USDT:USDT...
[539/547] Scanning XTZ/USDT:USDT...
[540/547] Scanning YFI/USDT:USDT...
[541/547] Scanning YGG/USDT:USDT...
[542/547] Scanning ZENT/USDT:USDT...
[543/547] Scanning ZETA/USDT:USDT...
[544/547] Scanning ZIL/USDT:USDT...
[545/547] Scanning ZK/USDT:USDT...
[546/547] Scanning ZRO/USDT:USDT...
[547/547] Scanning ZRX/USDT:USDT...
2025-10-10 10:34:04 UT